# Import des packages

In [2]:
import duckdb
import pandas as pd

# Chemin et données brutes

In [6]:
path_to_read = "../data/data_phyto/raw"
path_to_save = "../data/data_phyto/processed"

In [4]:
!whoami

guillaume


In [31]:
data_epci = duckdb.read_csv(f"{path_to_read}/data_epci.csv", delimiter=';', ignore_errors=True)
data_sau = duckdb.read_csv(f"{path_to_read}/sau_2025.csv")


## Fichier phyto 

### Phyto par commune avec sau

In [32]:
query = """ 
 SELECT
    geocode_epci,
    ROUND(TRY_CAST(valeur AS DOUBLE), 2) AS sau_ha
FROM data_sau
WHERE date_mesure > DATE '2010-12-31'
"""

data_sau = duckdb.sql(query)
data_sau

┌──────────────┬──────────┐
│ geocode_epci │  sau_ha  │
│   varchar    │  double  │
├──────────────┼──────────┤
│ 247400773    │  2228.34 │
│ 245501176    │ 15404.11 │
│ 200066785    │ 37332.64 │
│ 200040574    │  8968.44 │
│ 242010130    │  9759.43 │
│ 200071926    │ 27862.03 │
│ 200070670    │ 22086.67 │
│ 200033025    │ 16748.35 │
│ 200040798    │  3634.74 │
│ 247000664    │   5774.7 │
│     ·        │      ·   │
│     ·        │      ·   │
│     ·        │      ·   │
│ 200067767    │  9920.79 │
│ 200041762    │ 23892.37 │
│ 200058477    │   210.75 │
│ 200041861    │  19108.4 │
│ 200067064    │  7862.31 │
│ 240800862    │ 62670.81 │
│ 244301131    │   4687.1 │
│ 249500489    │  1769.72 │
│ 246700967    │  8109.07 │
│ 200041168    │  10541.1 │
├──────────────┴──────────┤
│  1256 rows (20 shown)   │
└─────────────────────────┘

In [33]:
data_phyto = duckdb.read_parquet(f"{path_to_read}/achat_commune_phyto.parquet")

In [7]:
q_join_epci_phyto = """
SELECT 
    phyto.annee,
    epci.dept,
    epci.siren,
    TRY_CAST(phyto.quantite_substance AS DOUBLE) AS quantite_substance
FROM data_phyto AS phyto
INNER JOIN data_epci AS epci
    ON epci.insee = phyto.code_insee
ORDER BY dept, annee, siren
"""

phyto = duckdb.sql(q_join_epci_phyto)
phyto

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────┬─────────┬───────────┬────────────────────┐
│  annee  │  dept   │   siren   │ quantite_substance │
│ varchar │ varchar │   int64   │       double       │
├─────────┼─────────┼───────────┼────────────────────┤
│ 2015    │ 01      │ 200029999 │              1.376 │
│ 2015    │ 01      │ 200029999 │              0.021 │
│ 2015    │ 01      │ 200029999 │              2.674 │
│ 2015    │ 01      │ 200029999 │               0.18 │
│ 2015    │ 01      │ 200029999 │               1.29 │
│ 2015    │ 01      │ 200029999 │              0.938 │
│ 2015    │ 01      │ 200029999 │              0.001 │
│ 2015    │ 01      │ 200029999 │              0.602 │
│ 2015    │ 01      │ 200029999 │              0.029 │
│ 2015    │ 01      │ 200029999 │             59.251 │
│  ·      │ ·       │     ·     │                ·   │
│  ·      │ ·       │     ·     │                ·   │
│  ·      │ ·       │     ·     │                ·   │
│ 2015    │ 01      │ 200040350 │              0.134 │
│ 2015    

### Phytorelative par epci

In [11]:
phyto

┌─────────┬─────────┬───────────┬────────────────────┐
│  annee  │  dept   │   siren   │ quantite_substance │
│ varchar │ varchar │   int64   │       double       │
├─────────┼─────────┼───────────┼────────────────────┤
│ 2015    │ 01      │ 200029999 │              0.049 │
│ 2015    │ 01      │ 200029999 │              0.028 │
│ 2015    │ 01      │ 200029999 │              1.804 │
│ 2015    │ 01      │ 200029999 │              0.198 │
│ 2015    │ 01      │ 200029999 │              0.393 │
│ 2015    │ 01      │ 200029999 │              0.002 │
│ 2015    │ 01      │ 200029999 │              0.382 │
│ 2015    │ 01      │ 200029999 │              2.815 │
│ 2015    │ 01      │ 200029999 │              0.064 │
│ 2015    │ 01      │ 200029999 │              0.041 │
│  ·      │ ·       │     ·     │                ·   │
│  ·      │ ·       │     ·     │                ·   │
│  ·      │ ·       │     ·     │                ·   │
│ 2015    │ 01      │ 200040350 │              1.017 │
│ 2015    

In [ ]:
#Requête pour obtenir la quantité totale de substance par département, par année et par EPCI
#Requête lourde, à optimiser plus tard si besoin
q_phyto_per_year = """ 
SELECT 
    annee,
    dept,
    siren,
    SUM(quantite_substance) AS total_quantite_substance
FROM phyto
GROUP BY annee, dept, siren
ORDER BY dept, annee, siren
"""

phyto_per_year = duckdb.sql(q_phyto_per_year)
phyto_per_year

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

IOException: IO Error: Could not write file ".tmp/duckdb_temp_block-4611686018427494457.block": No space left on device

In [8]:
data_phyto_relative = duckdb.read_csv(f"{path_to_save}/phyto_relative_csv.csv")
data_phyto_relative


┌───────────┬─────────┬─────────┬───────────────┐
│   siren   │ pop_tot │ dep_com │ total_relatif │
│   int64   │ varchar │ varchar │    double     │
├───────────┼─────────┼─────────┼───────────────┤
│ 200070852 │ 21 965  │ 01      │          1.06 │
│ 200070118 │ 21 470  │ 01      │          1.81 │
│ 240100750 │ 106 924 │ 01      │          0.89 │
│ 240100883 │ 84 625  │ 01      │          4.32 │
│ 200040590 │ 75 051  │ 01      │          6.09 │
│ 240100800 │ 25 583  │ 01      │           1.0 │
│ 200029999 │ 15 156  │ 01      │          0.96 │
│ 200042497 │ 41 975  │ 01      │          1.94 │
│ 200042935 │ 65 428  │ 01      │          0.21 │
│ 200069193 │ 41 461  │ 01      │          5.48 │
│     ·     │   ·     │ ·       │            ·  │
│     ·     │   ·     │ ·       │            ·  │
│     ·     │   ·     │ ·       │            ·  │
│ 249730045 │ 153 249 │ 973     │          0.07 │
│ 249740077 │ 185 848 │ 974     │          1.29 │
│ 249740101 │ 221 532 │ 974     │          0.32 │


In [17]:
q_phyto_relative = """ 
SELECT
    p2.dept,
    p1.siren,
    TRY_CAST(REPLACE(p1.pop_tot,' ','') AS DOUBLE) AS pop_tot,
    p1.total_relatif AS total_relatif,
    p2.total_sau2020
FROM data_phyto_relative AS p1
INNER JOIN df_sau_totale AS p2 ON p1.siren = p2.siren
ORDER BY p2.dept, p1.siren
"""

df_phyto_relative = duckdb.sql(q_phyto_relative)
df_phyto_relative.to_csv(f"{path_to_save}/phyto_relative_csv.csv")